In [1]:
from randomuser import RandomUser
import pandas as pd
import requests
import random
from IPython.utils import io
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from pathlib import Path
from dotenv import load_dotenv
import os
from googleapiclient.http import MediaFileUpload
import gspread
import json


In [2]:
"""
Tarefa 1

Usar a API do Random User Generator para gerar uma lista de 1.000 usuários brasileiros
"""

users = RandomUser.generate_users(1000, {"nat": "br"})

print(len(users))

1000


In [3]:
"""
Tarefa 1.1

Faça uma tabela com todas as informações desses usuários
"""

users_info = []
for user in users:
    user_info = {
        "first_name": user.get_first_name(),
        "last_name": user.get_last_name(),
        "gender": user.get_gender(),
        "email": user.get_email(),
        "cell": user.get_cell(),
        "age": user.get_age(),
    }
    users_info.append(user_info)

users_df = pd.DataFrame(users_info)

print(users_df.head().to_markdown())

|    | first_name   | last_name    | gender   | email                          | cell           |   age |
|---:|:-------------|:-------------|:---------|:-------------------------------|:---------------|------:|
|  0 | Estéfano     | da Mata      | male     | estefano.damata@example.com    | (56) 9693-3238 |    78 |
|  1 | Maricelda    | Moreira      | female   | maricelda.moreira@example.com  | (04) 7121-9919 |    36 |
|  2 | Miriam       | da Conceição | female   | miriam.daconceicao@example.com | (88) 0404-3545 |    63 |
|  3 | Alcides      | da Mata      | male     | alcides.damata@example.com     | (95) 9629-9084 |    40 |
|  4 | Karina       | Moreira      | female   | karina.moreira@example.com     | (07) 8418-9067 |    72 |


In [4]:
"""
Tarefa 1.2

Crie uma coluna que indique o Estado do usuário, baseado em seu telefone celular
Se o telefone celular tiver um DDD inexistente remova o usuário da tabela
"""


def get_state_from_ddd(ddd):
    state = {
        "11": "São Paulo",
        "12": "São Paulo",
        "13": "São Paulo",
        "14": "São Paulo",
        "15": "São Paulo",
        "16": "São Paulo",
        "17": "São Paulo",
        "18": "São Paulo",
        "19": "São Paulo",
        "21": "Rio de Janeiro",
        "22": "Rio de Janeiro",
        "24": "Rio de Janeiro",
        "27": "Espírito Santo",
        "28": "Espírito Santo",
        "31": "Minas Gerais",
        "32": "Minas Gerais",
        "33": "Minas Gerais",
        "34": "Minas Gerais",
        "35": "Minas Gerais",
        "37": "Minas Gerais",
        "38": "Minas Gerais",
        "41": "Paraná",
        "42": "Paraná",
        "43": "Paraná",
        "44": "Paraná",
        "45": "Paraná",
        "46": "Paraná",
        "47": "Santa Catarina",
        "48": "Santa Catarina",
        "49": "Santa Catarina",
        "51": "Rio Grande do Sul",
        "53": "Rio Grande do Sul",
        "54": "Rio Grande do Sul",
        "55": "Rio Grande do Sul",
        "61": "Distrito Federal",
        "62": "Goiás",
        "63": "Tocantins",
        "64": "Goiás",
        "65": "Mato Grosso",
        "66": "Mato Grosso",
        "67": "Mato Grosso do Sul",
        "68": "Acre",
        "69": "Rondônia",
        "71": "Bahia",
        "73": "Bahia",
        "74": "Bahia",
        "75": "Bahia",
        "77": "Bahia",
        "79": "Sergipe",
        "81": "Pernambuco",
        "82": "Alagoas",
        "83": "Paraíba",
        "84": "Rio Grande do Norte",
        "85": "Ceará",
        "86": "Piauí",
        "87": "Pernambuco",
        "88": "Ceará",
        "89": "Piauí",
        "91": "Pará",
        "92": "Amazonas",
        "93": "Pará",
        "94": "Pará",
        "95": "Roraima",
        "96": "Amapá",
        "97": "Amazonas",
        "98": "Maranhão",
        "99": "Maranhão",
    }
    return state.get(ddd, None)


for index, user_row in users_df.iterrows():
    ddd = user_row.cell[1:3]
    state = get_state_from_ddd(ddd)

    if state:
        users_df.loc[index, "state"] = state
    else:
        users_df.drop(index, inplace=True)

users_df.reset_index(drop=True, inplace=True)

print(users_df.head().to_markdown())

|    | first_name   | last_name    | gender   | email                          | cell           |   age | state               |
|---:|:-------------|:-------------|:---------|:-------------------------------|:---------------|------:|:--------------------|
|  0 | Miriam       | da Conceição | female   | miriam.daconceicao@example.com | (88) 0404-3545 |    63 | Ceará               |
|  1 | Alcides      | da Mata      | male     | alcides.damata@example.com     | (95) 9629-9084 |    40 | Roraima             |
|  2 | Marlene      | da Luz       | female   | marlene.daluz@example.com      | (35) 2801-6093 |    69 | Minas Gerais        |
|  3 | Aleixo       | Teixeira     | male     | aleixo.teixeira@example.com    | (84) 0725-5908 |    77 | Rio Grande do Norte |
|  4 | Hermes       | da Cruz      | male     | hermes.dacruz@example.com      | (18) 6176-8557 |    69 | São Paulo           |


In [5]:
"""
Tarefa 2

Baixe os Pokémons iniciais de todas as gerações por meio da PokeAPI
"""

initial_pokemons = [
    "bulbasaur",
    "charmander",
    "squirtle",
    "pikachu",
    "chikorita",
    "cyndaquil",
    "totodile",
    "treecko",
    "torchic",
    "mudkip",
    "turtwig",
    "chimchar",
    "piplup",
    "snivy",
    "tepig",
    "oshawott",
    "chespin",
    "fennekin",
    "froakie",
    "rowlet",
    "litten",
    "popplio",
    "grookey",
    "scorbunny",
    "sobble",
]

url = "https://pokeapi.co/api/v2/pokemon/"

pokemons = []
for pokemon_name in initial_pokemons:
    url_pokemon = url + f"{pokemon_name}"
    response = requests.get(url_pokemon)
    if response.status_code == 200:
        pokemons.append(response.json())
        print(f"{pokemon_name} - {response}")
    else:
        print(f"{pokemon_name} - {response}")

bulbasaur - <Response [200]>
charmander - <Response [200]>
squirtle - <Response [200]>
pikachu - <Response [200]>
chikorita - <Response [200]>
cyndaquil - <Response [200]>
totodile - <Response [200]>
treecko - <Response [200]>
torchic - <Response [200]>
mudkip - <Response [200]>
turtwig - <Response [200]>
chimchar - <Response [200]>
piplup - <Response [200]>
snivy - <Response [200]>
tepig - <Response [200]>
oshawott - <Response [200]>
chespin - <Response [200]>
fennekin - <Response [200]>
froakie - <Response [200]>
rowlet - <Response [200]>
litten - <Response [200]>
popplio - <Response [200]>
grookey - <Response [200]>
scorbunny - <Response [200]>
sobble - <Response [200]>


In [6]:
"""
Tarefa 2.1

Atribua um Pokémon inicial aleatório para cada um dos usuários
"""

for user_index, user_row in users_df.iterrows():
    users_df.loc[user_index, "pokemon"] = random.choice(initial_pokemons)

print(users_df.head().to_markdown())

|    | first_name   | last_name    | gender   | email                          | cell           |   age | state               | pokemon   |
|---:|:-------------|:-------------|:---------|:-------------------------------|:---------------|------:|:--------------------|:----------|
|  0 | Miriam       | da Conceição | female   | miriam.daconceicao@example.com | (88) 0404-3545 |    63 | Ceará               | squirtle  |
|  1 | Alcides      | da Mata      | male     | alcides.damata@example.com     | (95) 9629-9084 |    40 | Roraima             | squirtle  |
|  2 | Marlene      | da Luz       | female   | marlene.daluz@example.com      | (35) 2801-6093 |    69 | Minas Gerais        | scorbunny |
|  3 | Aleixo       | Teixeira     | male     | aleixo.teixeira@example.com    | (84) 0725-5908 |    77 | Rio Grande do Norte | snivy     |
|  4 | Hermes       | da Cruz      | male     | hermes.dacruz@example.com      | (18) 6176-8557 |    69 | São Paulo           | litten    |


In [7]:
"""
Tarefa 3.a

Conecte com o Google Drive
Disponível em: https://drive.google.com/drive/folders/1GvSdaQwIXeNE1KBhb-3Z-r-Sp_fCT2-E?usp=sharing
"""

SCOPES = ["https://www.googleapis.com/auth/drive"]
credentials_json = Path("..") / "credentials.json"
flow = InstalledAppFlow.from_client_secrets_file(credentials_json, SCOPES)
with io.capture_output() as captured:
    credentials = flow.run_local_server(port=0)
service = build("drive", "v3", credentials=credentials)

In [8]:
"""
Tarefa 3.b

Crie uma pasta para cada elemento dos Pokémons
"""

pokemon_types = []
for pokemon in pokemons:
    for type_item in pokemon["types"]:
        pokemon_types.append(type_item["type"]["name"])
pokemon_types = list(set(pokemon_types))


env_path = Path("..") / ".env"
load_dotenv(dotenv_path=env_path)

greenpeace_folder_id = os.environ.get("GOOGLE_DRIVE_GREENPEACE_FOLDER_ID")
try:
    files = service.files().list(q=f"'{greenpeace_folder_id}' in parents").execute()
except HttpError as err:
    print(err)

file_names_list = []
for file in files["files"]:
    file_names_list.append(file["name"])

google_drive_folder_id_by_type = {}
for pokemon_type in pokemon_types:
    if pokemon_type not in file_names_list:
        try:
            file_metadata = {
                "name": pokemon_type,
                "mimeType": "application/vnd.google-apps.folder",
                "parents": [greenpeace_folder_id],
            }
            file = service.files().create(body=file_metadata).execute()
            google_drive_folder_id_by_type[pokemon_type] = file.get("id")
            print(f"Folder for type *{pokemon_type}* created successfully with ID: {file.get('id')}")

        except HttpError as err:
            print(f"{err} for type *{pokemon_type}*")
    else:
        print(f"Folder for type *{pokemon_type}* already exists")

Folder for type *fire* created successfully with ID: 1JwSMdMlhH-8W-3IaG11mfFIRt654Spte
Folder for type *electric* created successfully with ID: 1kLYTUvBPcCp4qsoAmvwU3d_bYM_nNeaJ
Folder for type *water* created successfully with ID: 1VJtJViuSebZ8lSNYMEr2CaWD5dLrslWi
Folder for type *grass* created successfully with ID: 1v-tuhfVenofetZgZluym1RYts51mrAYM
Folder for type *flying* created successfully with ID: 19NQ0bYiau9s0R5aoU7RM-SRrxFl6lRpE
Folder for type *poison* created successfully with ID: 1A0QsgG1VbvbKmWU-WbwiXWVs1RLKt0mK


In [9]:
"""
Tarefa 3.1

Adicione em cada uma das pastas dois arquivos em CSV
Cada pasta deve ter dois arquivos um com todos os homens com Pokémons
daquele tipo, e outro com todas as mulheres com Pokémons daquele tipo
"""

pokemon_by_type = {pokemon_type: [] for pokemon_type in pokemon_types}

for pokemon in pokemons:
    for type_item in pokemon["types"]:
        for pokemon_type in pokemon_types:
            if type_item["type"]["name"] == pokemon_type:
                pokemon_by_type[pokemon_type].append(pokemon["name"])

genders = ["male", "female"]
for pokemon_type in pokemon_types:
    for gender in genders:
        filtered_users_df = users_df[users_df["pokemon"].isin(pokemon_by_type[pokemon_type])]
        filtered_users_df = filtered_users_df[filtered_users_df["gender"] == "male"]
        filtered_users_df.to_csv(f"csvs/{pokemon_type}_{gender}_users.csv", index=False)

        try:
            filename = f"{pokemon_type}_{gender}_users.csv"
            file_metadata = {
                "name": filename,
                "parents": [google_drive_folder_id_by_type[pokemon_type]],
            }
            media = MediaFileUpload(f"csvs/{filename}", mimetype="text/plain")
            file = service.files().create(body=file_metadata, media_body=media, fields="id").execute()
            print(
                f"File for type *{pokemon_type}* and gender *{gender}* created successfully with ID: {file.get('id')}"
            )

        except HttpError as err:
            print(f"{err} for type *{pokemon_type}* and gender *{gender}*")

File for type *fire* and gender *male* created successfully with ID: 14yNO8jzCbY8wxBHm3H4MCO3XS76oiAjF
File for type *fire* and gender *female* created successfully with ID: 1FGgiTu3qbvkhZb677QW8dvt_yvHOY76t
File for type *electric* and gender *male* created successfully with ID: 16bHx7IoqYGgWCj98Kaa3NqcT_KfGxsFA
File for type *electric* and gender *female* created successfully with ID: 14a4Tyhk0o93vUKilThltfRIUlkZFOGl9
File for type *water* and gender *male* created successfully with ID: 125_WB01mg_FFfVSwotUds7En51AFlyX1
File for type *water* and gender *female* created successfully with ID: 1yWUbGC6OXspIMwDC_Ssb0gdxTJ3UBFNS
File for type *grass* and gender *male* created successfully with ID: 1wHXnoEl0aMOi1cU0QD-XtIBo2PsRAZ4F
File for type *grass* and gender *female* created successfully with ID: 1hx6hAuwbFOhT2WgTc3skImmdNlcnh9vN
File for type *flying* and gender *male* created successfully with ID: 1VNVzfwMRY92UMosRaFVSEstBGo7MuQKt
File for type *flying* and gender *female* created 

In [10]:
"""
Tarefa 4

Por fim, crie um Google Sheets com um resumo
Disponível em: https://docs.google.com/spreadsheets/d/1xnxHHwzIrMScqnTYfiZdQ1_1rUAFCvPXhWhq1a5nFlk/edit?usp=sharing
"""

credentials_json = Path("..") / "credentials.json"
credentials_dict = json.load(open(credentials_json))
with io.capture_output() as captured:
    gc, authorized_user = gspread.oauth_from_dict(credentials_dict)

sh = gc.create("[GREENPEACE] Resumo")

worksheet_1 = sh.add_worksheet(title="Número de pessoas por região do Brasil", rows=100, cols=20)
worksheet_2 = sh.add_worksheet(title="Elemento de pokemons por região do Brasil", rows=100, cols=20)

sh.del_worksheet(sh.worksheet("Página1"))

{'spreadsheetId': '1xnxHHwzIrMScqnTYfiZdQ1_1rUAFCvPXhWhq1a5nFlk',
 'replies': [{}]}

In [11]:
"""
Tarefa 4.a

Indique o número de pessoas por região do Brasil (Norte, Sul, Sudeste, Nordeste, Centro-Oeste)
"""

regions = {
    "Norte": [
        "Amazonas",
        "Acre",
        "Tocantins",
        "Rondônia",
        "Pará",
        "Roraima",
        "Amapá",
    ],
    "Sul": [
        "Rio Grande do Sul",
        "Santa Catarina",
        "Paraná",
        "Rio Grande do Sul",
    ],
    "Sudeste": [
        "São Paulo",
        "Rio de Janeiro",
        "Minas Gerais",
        "Espírito Santo",
    ],
    "Nordeste": [
        "Pernambuco",
        "Bahia",
        "Ceará",
        "Rio Grande do Norte",
        "Paraíba",
        "Maranhão",
        "Piauí",
        "Sergipe",
        "Alagoas",
    ],
    "Centro-Oeste": [
        "Goiás",
        "Mato Grosso",
        "Mato Grosso do Sul",
        "Distrito Federal",
    ],
}


def get_region(state):
    for region, states in regions.items():
        if state in states:
            return region


def get_pokemon_types(pokemon):
    pokemon_types = []
    for pokemon_type, pokemon_list in pokemon_by_type.items():
        if pokemon in pokemon_list:
            pokemon_types.append(pokemon_type)
    return pokemon_types


extended_users_df = users_df.copy()
extended_users_df["region"] = users_df["state"].apply(get_region)
users_by_region_df = extended_users_df["region"].value_counts().to_frame()
print(users_by_region_df.to_markdown())

users_by_region_df.reset_index(inplace=True)
users_by_region_df.columns = ["Região", "Número de pessoas"]
worksheet_1.update([users_by_region_df.columns.values.tolist()] + users_by_region_df.values.tolist())


| region       |   count |
|:-------------|--------:|
| Sudeste      |     211 |
| Nordeste     |     158 |
| Sul          |     138 |
| Norte        |     111 |
| Centro-Oeste |      58 |


{'spreadsheetId': '1xnxHHwzIrMScqnTYfiZdQ1_1rUAFCvPXhWhq1a5nFlk',
 'updatedRange': "'Número de pessoas por região do Brasil'!A1:B6",
 'updatedRows': 6,
 'updatedColumns': 2,
 'updatedCells': 12}

In [12]:
"""
Tarefa 4.b

Indique o número de elementos de Pokemon por região do Brasil
"""


types_by_region = {}
for region in regions.keys():
    types_by_region[region] = {}
    for pokemon_type in pokemon_types:
        types_by_region[region][pokemon_type] = 0

for index, row in users_df.iterrows():
    for pokemon_type in get_pokemon_types(row.pokemon):
        types_by_region[get_region(row.state)][pokemon_type] += 1

types_by_region_df = pd.DataFrame(types_by_region)
print(types_by_region_df.head().to_markdown())

types_by_region_df.index.name = "Elemento"
types_by_region_df.reset_index(inplace=True)
worksheet_2.update([types_by_region_df.columns.values.tolist()] + types_by_region_df.values.tolist())


|          |   Norte |   Sul |   Sudeste |   Nordeste |   Centro-Oeste |
|:---------|--------:|------:|----------:|-----------:|---------------:|
| fire     |      23 |    37 |        69 |         43 |             25 |
| electric |       6 |    10 |         9 |          8 |              2 |
| water    |      40 |    38 |        66 |         49 |             22 |
| grass    |      42 |    53 |        67 |         58 |              9 |
| flying   |       8 |     4 |         9 |          5 |              2 |


{'spreadsheetId': '1xnxHHwzIrMScqnTYfiZdQ1_1rUAFCvPXhWhq1a5nFlk',
 'updatedRange': "'Elemento de pokemons por região do Brasil'!A1:F7",
 'updatedRows': 7,
 'updatedColumns': 6,
 'updatedCells': 42}